## Importing required libraries

In [1]:
from os import stat
from networkx.algorithms.components.connected import is_connected
from networkx.classes.function import neighbors
from networkx.linalg.algebraicconnectivity import fiedler_vector
import scipy as sp
import networkx as nx
from scipy.io import mmread
from scipy.sparse.coo import coo_matrix
from scipy.sparse.linalg import eigs
from numpy import linalg as LA
import numpy as np
import csv
import statistics

## Creating Graph object

In [16]:
class Graph:
    def __init__(self, sparse):
        self.graph = nx.from_scipy_sparse_matrix(sparse)
        self.adj = nx.adjacency_matrix(self.graph)
        self.laplacian = nx.laplacian_matrix(self.graph)
    def degree_centrality(self):
        return nx.degree_centrality(self.graph)
    def closeness_centrality(self):
        return nx.closeness_centrality(self.graph)
    def closeness_centrality_node(self, node):
        return nx.closeness_centrality(self.graph, node)
    def betweenness_centrality(self):
        return nx.betweenness_centrality(self.graph, k = min(self.graph.number_of_nodes() , 500))
    def eigenvector_centrality(self):
        return nx.eigenvector_centrality(self.graph)
    def is_connected(self):
        return nx.is_connected(self.graph)
    def lfvc(self):
        if (not self.is_connected()): return "Not possible"
        fiedler_vector = nx.fiedler_vector(self.graph)
        lfvclist = [sum((fiedler_vector[j]-fiedler_vector[i[0]])*(fiedler_vector[j]-fiedler_vector[i[0]]) for j in self.graph.neighbors(i[0])) for i in self.graph.nodes(data = True)]
        return lfvclist
    def lfvc_node(self, node):
        if (not self.is_connected()): return "Not possible"
        nodes = list(self.graph.nodes(data = True))
        n = nodes[node]
        fiedler_vector = nx.fiedler_vector(self.graph)
        fiedler = fiedler_vector[n[0]]
        lfvcthis = sum((fiedler_vector[j]-fiedler)*(fiedler_vector[j]-fiedler) for j in self.graph.neighbors(n[0]))
        return lfvcthis
    def neighbourhood_hopset(self, index, k = 10):
        nbrs = set([index])
        for l in range(k):
            nbrs = set((nbr for nbr in self.graph[n] for n in nbrs))
        return len(nbrs)
    def clustering_coefficient(self):
        return nx.clustering(self.graph)
    def clustering_coefficient_node(self, node):
        return nx.clustering(self.graph, node)
    def ego_centrality_node(self, node):
        g = nx.ego_graph(self.graph, node)
        nodes = list(g.nodes(data = True))
        n = node
        for i in nodes:
            if i[0] == node:
                n = i
                break
        centrality =  nx.betweenness_centrality(g)
        return centrality[node]
    def nodes_of_interest(self):
        l = list(nx.degree_centrality(self.graph))
        mean = statistics.mean(l)
        median = statistics.median_high(l)
        closest_mean = min(l, key = lambda x:abs(x-mean))
        max_value = max(l)
        min_value = min(l)
        return l.index(median), l.index(closest_mean), l.index(min_value), l.index(max_value)

## Initialising and creating instances of graph object using different *.mtx files

In [3]:
karate = mmread('../assets/S_soc-karate.mtx').astype(np.int8).tocsr()
webedu = mmread('../assets/M_web-edu.mtx').astype(np.int8).tocsr()
internet = mmread('../assets/L_tech-internet-as.mtx').astype(np.int8).tocsr()

In [4]:
G1 = Graph(karate)
G2 = Graph(webedu)
G3 = Graph(internet)
print(("-"*50)+"Graphs made"+("-"*50))

--------------------------------------------------Graphs made--------------------------------------------------


In [5]:
# Checking if instantiated graphs are connected

c1 = G1.is_connected()
c2 = G2.is_connected()
c3 = G3.is_connected()
print(f'G1 is connected: {c1}')
print(f'G2 is connected: {c2}')
print(f'G3 is connected: {c3}')

G1 is connected: True
G2 is connected: True
G3 is connected: True


## Finding Centralities

In [ ]:
# EGO centrality
# print(G.ego_centrality_node(4))
# print("ego graph made")

In [13]:
# Finding lfvc node

lfvc1 = G1.lfvc_node(0)
lfvc2 = G2.lfvc_node(0)
# lfvc3 = G3.lfvc_node(0)
print(lfvc1)
print(lfvc2)
# print(lfvc3)

0.27966447622970536
0.0036321147457527595


In [14]:
# Finding nodes of interest
print("Nodes of interest: ")
print(G1.nodes_of_interest())
print(G2.nodes_of_interest())
print(G3.nodes_of_interest())

Nodes of interest: 
(17, 16, 0, 33)
(1515, 1515, 0, 3030)
(20082, 20081, 0, 40163)


In [17]:
# Finding Centralities of smallest size graph, i.e. soc-karate

print("soc-karate :")
dc1 = G1.degree_centrality()
cc1 = G1.closeness_centrality()
bc1 = G1.betweenness_centrality()
ec1 = G1.eigenvector_centrality()
clc1 = G1.clustering_coefficient_node(0)
lfvc_val1 = G1.lfvc()
nhc1 = G1.neighbourhood_hopset(0,2)
print(("-"*100))
print("lfvc")
print(lfvc_val1)
print(("-"*100))
print("degree centrality")
print(dc1)
print(("-"*100))
print("closeness centrality")
print(cc1)
print(("-"*100))
print("betweenness centrality")
print(bc1)
print(("-"*100))
print("eigenvector centrality")
print(ec1)
print(("-"*100))
print("Clusters of node 1")
print(clc1)
print(("-"*100))
print("neighbouring hopset")
print(nhc1)
print(("-"*100))


soc-karate :
----------------------------------------------------------------------------------------------------
lfvc
[0.2796644791203452, 0.031092731488145446, 0.06907300617062377, 0.014187972342677699, 0.03127549599301062, 0.05610939736946621, 0.056109397369096224, 0.009428901110558843, 0.03882565659531453, 0.005522978165210318, 0.03127549599299592, 0.00977238955703561, 0.002919005845701989, 0.031153835533104093, 0.002972709590104008, 0.002972709590103589, 0.019617106574343984, 0.003611393697034108, 0.0029727095901053834, 0.028033824998073976, 0.002972709590103927, 0.003611393697033781, 0.002972709590104143, 0.0029525819012913195, 0.003651674334022202, 0.003960777081110232, 0.005030760214384285, 0.012414442641018457, 0.005751480877862159, 0.004289528083000577, 0.018948645692554535, 0.05270747372861378, 0.029321909446177942, 0.061873169832481915]
----------------------------------------------------------------------------------------------------
degree centrality
{0: 0.48484848484848

In [9]:
# Finding Centralities of medium size graph, i.e. web-edu

print("web-edu :")
dc2 = G2.degree_centrality()
cc2 = G2.closeness_centrality()
bc2 = G2.betweenness_centrality()
ec2 = G2.eigenvector_centrality()
clc2 = G2.clustering_coefficient_node(0)
lfvc_val2 = G2.lfvc()
nhc2 = G2.neighbourhood_hopset(0,2)
print(("-"*100))
print("lfvc")
print(lfvc_val2)
print(("-"*100))
print("degree centrality")
print(dc2)
print(("-"*100))
print("closeness centrality")
print(cc2)
print(("-"*100))
print("betweenness centrality")
print(bc2)
print(("-"*100))
print("eigenvector centrality")
print(ec2)
print(("-"*100))
print("Clusters of node 1")
print(clc2)
print(("-"*100))
print("neighbouring hopset")
print(nhc2)
print(("-"*100))

web-edu :
----------------------------------------------------------------------------------------------------
lfvc
[0.003632114329712128, 1.668961313600583e-08, 4.169616937033348e-06, 1.6689613136005455e-08, 1.6689613136005557e-08, 1.6689613136005766e-08, 1.668961313600559e-08, 1.6689613136006074e-08, 1.668961313600585e-08, 1.668961313600572e-08, 1.668961313600588e-08, 4.3940193325384594e-15, 1.088628769486026e-10, 4.393935846464154e-15, 2.5224616389430475e-06, 0.00253734097466177, 4.618987640548357e-06, 2.6001263697291473e-06, 1.0722743118321219e-08, 1.0722743118321791e-08, 4.967713961259267e-07, 1.0722743118320956e-08, 1.0722743118320517e-08, 1.0722743118320618e-08, 1.0722743118320865e-08, 1.0722743118319708e-08, 1.0722743118320266e-08, 1.0722743118319781e-08, 1.0722743118320832e-08, 1.072274311832028e-08, 1.0722743118320656e-08, 4.393935846464154e-15, 4.393935846464154e-15, 4.393935846464154e-15, 5.453917407382564e-11, 5.4539174073812823e-11, 5.453917407399218e-11, 5.45391740739281

In [ ]:
# Finding Centralities of largest size graph, i.e. tech-internet-as

print("tech-internet-as :")
dc3 = G3.degree_centrality()
cc3 = G3.closeness_centrality()
bc3 = G3.betweenness_centrality()
ec3 = G3.eigenvector_centrality()
clc3 = G3.clustering_coefficient_node(0)
lfvc_val3 = G3.lfvc()
nhc3 = G3.neighbourhood_hopset(0,2)
print(("-"*100))
print("lfvc")
print(lfvc_val3)
print(("-"*100))
print("degree centrality")
print(dc3)
print(("-"*100))
print("closeness centrality")
print(cc3)
print(("-"*100))
print("betweenness centrality")
print(bc3)
print(("-"*100))
print("eigenvector centrality")
print(ec3)
print(("-"*100))
print("Clusters of node 1")
print(clc3)
print(("-"*100))
print("neighbouring hopset")
print(nhc3)
print(("-"*100))